<a href="https://colab.research.google.com/github/GenAIHub/genai-workshop/blob/main/02_RAG/02_ingestion_and_query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install boto3
!pip install pydantic==1.10.9
!pip installl lama_index
!pip install llama-index-embeddings-bedrock==0.1.4
!pip install llama-index-llms-bedrock==0.1.5
!pip install llama-index-vector-stores-postgres==0.1.5
!pip install langchain==0.0.333
!pip install PyMuPDF
!pip install llama-index-readers-file

In [4]:
import fitz

doc = fitz.open("./data/test.pdf")
full_pdf_content = ""
for page in range(doc.page_count):
  pg = doc.load_page(page)
  text = pg.get_text()
  full_pdf_content += text

print(full_pdf_content)

Master Outsourcing Agreement - Motorola
Inc. and Linuxcare Inc.
MASTER OUTSOURCING AGREEMENT
(MAINTENANCE SERVICES)
This Master Outsourcing Agreement (Maintenance Services) (the "Agreement")
is made as of the 13th day of December, 1999 by and between Motorola, Inc., a
Delaware corporation, through its Motorola Computer Group, with its principal
office located at 2900 South Diablo Way, Tempe, AZ 85282 ("MCG"), and LinuxCare,
Inc, with its principal office located at 650 Townsend St. San Francisco, CA
94103 (VENDOR).
RECITALS
A. MCG manufactures, sells, distributes and licenses certain computer hardware
and software products and other third party products in connection with such
computer products.
B. MCG provides installation, maintenance and other services for such computer
products and such third party products.
C. VENDOR is in the business of providing engineering support, training,
development, and consulting services for Linux products.
D. MCG desires to establish a contractual rela

In [9]:
from llama_index.core.node_parser.text.sentence import SentenceSplitter
from llama_index.core.schema import Document
import pprint as pp

chunk_size = 1024
chunk_overlap = 100

splitter = SentenceSplitter(
        chunk_size=chunk_size,
        # chunk_size=512,
        chunk_overlap=chunk_overlap,
    )

docs = splitter.get_nodes_from_documents(
        [
            Document(
                text=full_pdf_content,
                metadata={
                    "page": 0,
                    #"file_name": f"{metadata_file_name}",
                    "source": "text",
                    "document_id": f"some_id",
                },
                excluded_embed_metadata_keys=["page", "source", "document_id"],
                excluded_llm_metadata_keys=["page", "source", "document_id"],
            )
        ]
    )
pp.pprint(docs[0])

TextNode(id_='4c3a92cf-fbce-450c-ac89-073a643029a5', embedding=None, metadata={'page': 0, 'source': 'text', 'document_id': 'some_id'}, excluded_embed_metadata_keys=['page', 'source', 'document_id'], excluded_llm_metadata_keys=['page', 'source', 'document_id'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='b84330e6-de90-4777-a744-565f07d89d50', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page': 0, 'source': 'text', 'document_id': 'some_id'}, hash='ed0f15074af698f7161ac0db31d74d77e8a1e1cf11c5e44a091ce0f8bfd9fefe'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='5a9c67da-11f8-4482-a29c-23b511e7f43f', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='030d47558485b58f8a4659baddc595976c6c1c33773a67ac1885cb1d1c939fc7')}, text='Master Outsourcing Agreement - Motorola\nInc. and Linuxcare Inc.\nMASTER OUTSOURCING AGREEMENT\n(MAINTENANCE SERVICES)\nThis Master Outsourcing Agreement (Maintenance Services) (the "Agreement")\nis made as of the 13th day of De

In [11]:
from llama_index.embeddings.bedrock import BedrockEmbedding
from llama_index.llms.bedrock import Bedrock
from llama_index.core.llms import ChatMessage
from llama_index.core.settings import Settings
import boto3
import asyncio
import os


os.environ["AWS_REGION"] = "us-east-1"
os.environ["AWS_ACCESS_KEY_ID"] = ""
os.environ["AWS_SECRET_ACCESS_KEY"] = ""
#os.environ["AWS_SESSION_TOKEN"] = ""

embed_model_id = "amazon.titan-embed-text-v1"
llm_model_id = "anthropic.claude-3-haiku-20240307-v1:0"

BEDROCK_CLIENT = boto3.client("bedrock-runtime", "us-east-1")

llm = Bedrock(
    model=llm_model_id,
    client=BEDROCK_CLIENT,
    aws_region_name="us-east-1",
    temperature=0.1,
    max_tokens=512
    )


embed_model = BedrockEmbedding(model=embed_model_id, client=BEDROCK_CLIENT)

Settings.llm = llm
Settings.embed_model = embed_model

In [12]:
from llama_index.core.ingestion.pipeline import IngestionPipeline
from llama_index.core.extractors import QuestionsAnsweredExtractor
import asyncio

pipeline = IngestionPipeline(
          transformations=[
              QuestionsAnsweredExtractor(questions=2)
           ]
      )
nodes = await pipeline.arun(nodes = docs)
pp.pprint(nodes[0])

100%|██████████| 20/20 [01:07<00:00,  3.40s/it]

TextNode(id_='4c3a92cf-fbce-450c-ac89-073a643029a5', embedding=None, metadata={'page': 0, 'source': 'text', 'document_id': 'some_id', 'questions_this_excerpt_can_answer': 'Based on the provided context, here are two questions that can be answered using the information given:\n\n1. What is the purpose of this Master Outsourcing Agreement between Motorola, Inc. (MCG) and LinuxCare, Inc. (VENDOR)?\n\nThe context indicates that the purpose of this agreement is for VENDOR to provide a significant portion of MCG\'s support and other support-related services for MCG\'s computer hardware and software products, including third-party products. Specifically, VENDOR will provide engineering support, training, development, and consulting services for Linux products.\n\n2. What are the key definitions and terms outlined in this agreement?\n\nThe context provides several key definitions and terms, including:\n- "Base Term" - the duration of the agreement\n- "Customer Data" - the customer contract fil

In [13]:
from llama_index.core import VectorStoreIndex

index =  VectorStoreIndex(nodes)

print(index)

In [29]:
from llama_index.core import PromptTemplate
from IPython.display import Markdown, display

# define prompt viewing function
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}<br>" f"**Text:** <br>"
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown("<br><br>"))


qa_prompt_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question: {query_str}\n"
)

text_qa_template = PromptTemplate(qa_prompt_str)

query_engine = index.as_query_engine(
    similarity_top_k=2
    )

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": text_qa_template}
)

prompts_dict = query_engine.get_prompts()
display_prompt_dict(prompts_dict)

**Prompt Key**: response_synthesizer:text_qa_template<br>**Text:** <br>

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the question: {query_str}



<br><br>

**Prompt Key**: response_synthesizer:refine_template<br>**Text:** <br>

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


<br><br>

In [30]:
question = "What is this story about?"
response = query_engine.query(question)
print(response)

Based on the provided context, this document appears to be about the Motorola Code of Conduct. The key points are:

1. The Motorola Code of Conduct outlines the company's policies and standards of ethical conduct for its employees. It emphasizes the importance of integrity in Motorola's dealings with customers, suppliers, and governments.

2. The Code prohibits the use of Motorola's funds and assets for illegal payments, gifts/gratuities to business partners, and political contributions. It sets clear guidelines around these prohibited activities.

3. The Code states that adherence to its principles is a condition of employment at Motorola, and it will be administered and enforced throughout the company.

4. The document also describes the operating procedures for reporting and addressing potential violations of the Code, including the role of Motorola's General Counsel in providing advice, investigating issues, and determining if violations have occurred.

In summary, this appears to 